In [ ]:
import gymnasium as gym

from stable_baselines3 import PPO
from stable_baselines3.common.env_util import make_vec_env

# Parallel environments
vec_env = make_vec_env("CartPole-v1",seed=0, n_envs=2)

model = PPO("MlpPolicy", vec_env, verbose=1)
model.learn(total_timesteps=25000)
model.save("ppo_agar_io")

del model # remove to demonstrate saving and loading

model = PPO.load("ppo_agar_io")

obs = vec_env.reset()
while True:
    action, _states = model.predict(obs)
    print(_states.shape)
    # obs, rewards, dones, info = vec_env.step(action)
    # vec_env.render("human")

In [4]:
import numpy as np

In [25]:
#convert observations to numpy array
c=[[0.24722777193253823, 1.0864478241664053], [11.0, 0.0], [22.0, 0.0], [False]]

In [30]:
c_array = np.array(c, dtype=float)

In [31]:
c_array 

array([ 0.24722777,  1.08644782, 11.        ,  0.        , 22.        ,
        0.        ,  0.        ])

In [29]:
c=*c

SyntaxError: can't use starred expression here (3068832564.py, line 1)

In [26]:
c = [item for sublist in c for item in sublist]
c = np.asarray(c)

In [27]:
c

array([ 0.24722777,  1.08644782, 11.        ,  0.        , 22.        ,
        0.        ,  0.        ])

In [5]:
obs2rl=np.zeros(16)
a=[1,2,3,4]
b=[1,2,3,4]

In [8]:
for i in range (0,len(obs2rl)):
    obs2rl[i]=a

SyntaxError: invalid syntax (1366516897.py, line 1)

In [ ]:

obs2rl[:]=np.zeros()+observations_short[0][:],observations_short[1][:], observations_short[2][:], observations_short[3]

In [1]:
# from pyglet.window import key
import numpy as np
from agar.Env import AgarEnv
import time
import torch
import torch.nn as nn
import torch.optim as optim
render = False
num_agents = 1


class Args:
    def __init__(self):

        self.num_controlled_agent = num_agents
        self.num_processes = 64
        self.action_repeat = 1
        self.total_step = 1e8
        self.r_alpha = 0.1
        self.r_beta = 0.1
        self.seed = 42
        self.gamma = 0.99
        self.eval = True

def calculate_reward(observations):
    total_rewards = []
    for agent in observations:
        if observations[agent]["metadata"]["is_killed"]:
            observations[agent] = 0
            continue
        reward = 0
        for food in observations[agent]["food"]:
            reward += 1 / food["relative_position_x^2 + relative_position_y^2"]
        total_rewards.append(reward)
    return total_rewards


In [2]:
class Memory:
    def __init__(self):
        self.states = []
        self.actions = []
        self.rewards = []
        self.logprobs = []
        self.values = []
        self.next_value = None

    def clear(self):
        del self.states[:]
        del self.actions[:]
        del self.rewards[:]
        del self.logprobs[:]
        del self.values[:]
        del self.masks[:]
        self.next_value = None

In [3]:
class TransformerModel(nn.Module):
    def __init__(self,input_dim, dim_hidden, output_dim):
        super().__init__()
        self.fc1 = torch.nn.Sequential(
            torch.nn.Linear(input_dim, dim_hidden),
            torch.nn.ReLU(),
            torch.nn.Linear(dim_hidden, dim_hidden)
        )
        self.transformer = nn.TransformerEncoderLayer(d_model=dim_hidden, nhead=8)
        self.fc2 = torch.nn.Sequential(
            torch.nn.Linear(dim_hidden, output_dim)
        )
    def forward(self, x):
        out = self.fc1(x)
        out = out.permute(1, 0, 2)
        out = self.transformer(out)
        out = out.permute(1, 0, 2)
        out = self.fc2(out)
        return out

In [4]:
# i need to learn stohaistic policy gradient
agent_transformer = TransformerModel(578, 512, 3)
critic_transformer = TransformerModel(578, 512, 1)